In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)
types = {'id': int, 'ciudad':str, 'provincia': str}

training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','provincia','ciudad','idzona','metrostotales','precio'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','provincia','ciudad','idzona','lat','lng'])

In [2]:
training_data.precio.mean()/training_data.metrostotales.mean()

14317.519170109566

In [3]:
metros_ciudad = training_data.groupby('ciudad').agg({'metrostotales':'mean'}).reset_index().rename(columns={'metrostotales':'mts_ciudad'})

meanmts=metros_ciudad.mts_ciudad.mean()
metros_ciudad.mts_ciudad.fillna(meanmts,inplace=True)

training_data = pd.merge(training_data,metros_ciudad, on='ciudad', how='right')

In [4]:
training_data.metrostotales.fillna(training_data.mts_ciudad,inplace=True)
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239628 entries, 0 to 239627
Data columns (total 7 columns):
id               239628 non-null int32
ciudad           239628 non-null object
provincia        239628 non-null object
metrostotales    239628 non-null float64
idzona           211379 non-null float64
precio           239628 non-null float64
mts_ciudad       239628 non-null float64
dtypes: float64(4), int32(1), object(2)
memory usage: 13.7+ MB


In [5]:
training_data.idzona.fillna(-1, inplace=True)
props_zone = ((training_data.idzona.value_counts()>5).to_frame().reset_index()).rename(columns={'index':'idzona','idzona':'+5props'})
props_zone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 2 columns):
idzona     19720 non-null float64
+5props    19720 non-null bool
dtypes: bool(1), float64(1)
memory usage: 173.4 KB


In [6]:
training_data.ciudad.fillna('no_city', inplace=True)

In [7]:
precio_ciudad = training_data.groupby('ciudad').agg({'precio':'sum','metrostotales':'sum'}).reset_index()
precio_ciudad['precio_mts_ciudad'] = precio_ciudad.precio/precio_ciudad.metrostotales
precio_ciudad.drop(columns={'metrostotales','precio'},inplace=True)
precio_ciudad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 2 columns):
ciudad               875 non-null object
precio_mts_ciudad    875 non-null float64
dtypes: float64(1), object(1)
memory usage: 13.8+ KB


In [8]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239628 entries, 0 to 239627
Data columns (total 7 columns):
id               239628 non-null int32
ciudad           239628 non-null object
provincia        239628 non-null object
metrostotales    239628 non-null float64
idzona           239628 non-null float64
precio           239628 non-null float64
mts_ciudad       239628 non-null float64
dtypes: float64(4), int32(1), object(2)
memory usage: 13.7+ MB


In [9]:
training_data = pd.merge(training_data,props_zone, on='idzona', how='right')
training_data = pd.merge(training_data,precio_ciudad, on= 'ciudad', how='right')
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239628 entries, 0 to 239627
Data columns (total 9 columns):
id                   239628 non-null int32
ciudad               239628 non-null object
provincia            239628 non-null object
metrostotales        239628 non-null float64
idzona               239628 non-null float64
precio               239628 non-null float64
mts_ciudad           239628 non-null float64
+5props              239628 non-null bool
precio_mts_ciudad    239628 non-null float64
dtypes: bool(1), float64(5), int32(1), object(2)
memory usage: 15.8+ MB


In [16]:
training_data.tail()

,id,idzona
239995,119879,53666.0
239996,259178,51954.0
239997,131932,50003995.0
239998,146867,24162.0
239999,121958,23750.0


In [11]:
precio_zona = training_data.groupby('idzona').agg({'precio':'sum','metrostotales':'sum','+5props':'median','precio_mts_ciudad':'mean'}).reset_index()
precio_zona.loc[precio_zona['+5props'], 'precio_por_metro'] = precio_zona.precio/precio_zona.metrostotales
precio_zona.loc[~(precio_zona['+5props']), 'precio_por_metro'] = precio_zona.precio_mts_ciudad
precio_zona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 6 columns):
idzona               19720 non-null float64
precio               19720 non-null float64
metrostotales        19720 non-null float64
+5props              19720 non-null bool
precio_mts_ciudad    19720 non-null float64
precio_por_metro     19720 non-null float64
dtypes: bool(1), float64(5)
memory usage: 789.6 KB


In [12]:
precio_zona = precio_zona[['idzona','precio_por_metro']]

In [13]:
precio_zona.head()

,idzona,precio_por_metro
0,-1.0,12110.166717
1,22.0,25368.514320
2,23.0,8060.561494
3,26.0,5430.679406
4,27.0,8758.432621


In [195]:
###precio_zona.to_csv('../../res/ftr/precio_por_metro_idzona.csv')

In [48]:
training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','idzona','ciudad'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','idzona','ciudad'])

training_data.idzona.fillna(-1, inplace=True)
evaluation_data.idzona.fillna(-1, inplace=True)


In [49]:
evaluation_data=pd.merge(evaluation_data,precio_zona,on='idzona',how='left')
evaluation_data=pd.merge(evaluation_data,precio_ciudad,on='ciudad',how='left')

evaluation_data.precio_mts_ciudad.fillna(evaluation_data.precio_mts_ciudad.mean(),inplace=True)
evaluation_data.precio_por_metro.fillna(evaluation_data.precio_mts_ciudad,inplace=True)


training_data=pd.merge(training_data,precio_zona,on='idzona',how='left')
training_data=pd.merge(training_data,precio_ciudad,on='ciudad',how='left')

training_data.precio_mts_ciudad.fillna(training_data.precio_mts_ciudad.mean(),inplace=True)
training_data.precio_por_metro.fillna(training_data.precio_mts_ciudad,inplace=True)

In [50]:
training_data = training_data[['id','precio_por_metro']]
evaluation_data = evaluation_data[['id','precio_por_metro']]

In [53]:
training_data.head()

,id,precio_por_metro
0,254099,23256.837144
1,53461,15614.908380
2,247984,8441.101152
3,209067,9857.301547
4,185997,13804.584787


In [54]:
training_data.to_csv('../../res/ftr/precio_por_idzona_train.csv')
evaluation_data.to_csv('../../res/ftr/precio_por_idzona_evaluation.csv')